## 爬蟲設定

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.action_builder import ActionBuilder
import time
import pandas as pd


def fb_scrap_2(url, name, minu):
    posts_text = ['']
    SCROLL_PAUSE_TIME = 1
    scroll=True

    try:
        start_time = time.time()
        max_duration = minu * 60
        scroll = True
        while scroll:  # Adjust the range for more scrolls
            elapsed_time = time.time() - start_time
            if elapsed_time > max_duration:
                print("已達執行時間限制，停止爬取")
                break
#            current_scroll_position = driver.execute_script("return window.scrollY + window.innerHeight;")
#            page_height = driver.execute_script("return document.body.scrollHeight;")
#            if current_scroll_position >= page_height:
#                print("已經滾動到頁面最底部")
#                break

            # Find all post elements
            posts = driver.find_elements(By.XPATH, "//div[@data-ad-preview='message']")
        
            for post in posts:
                try:
               # Look for the "查看更多" button within each post
                    see_more = post.find_element(By.XPATH, ".//div[@role='button' and text()='查看更多']")
                    driver.execute_script("arguments[0].click();", see_more)
                    time.sleep(0.1)  # Wait for the full text to load
                except Exception as e:
                    pass

            opened_posts = driver.find_elements(By.XPATH, "//div[@data-ad-preview='message']")
            
            for post in opened_posts:
           #print(post)
                text = post.text
                if text not in posts_text:  # Avoid duplicates
                    posts_text.append(text)

       # Scroll down to load more posts
       # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            driver.execute_script("window.scrollBy(0, 2000);")
            time.sleep(SCROLL_PAUSE_TIME)

# except Exception as e:
#     print(e)

    except KeyboardInterrupt:
   # Print out collected posts
       for i, post in enumerate(posts_text, start=0):
           print(f"Post {i}: {post}\n{'-' * 40}")

    return {'name': name, 'text': posts_text}



## 爬蟲

In [ ]:

# 輸入名字 line33 改檔名 line128
# 時間限制 line54 預設60min 但大概20多分鐘爬到1000篇左右就會page crash停下來
# 如果覺得爬太久遠了要停直接把視窗關掉就好 一樣會輸出結果
# 點擊粉專 line41 那段如果他不聽話可以整段刪掉自己點 如果時間不夠可以把前段的time.sleep改長
# 中間如果有error只要他還有在爬應該就沒事（吧？ 輸出error主要是想看他最後是為什麼停下來而已

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.action_builder import ActionBuilder
import time
import pandas as pd

# 初始化 Chrome 瀏覽器
driver = webdriver.Chrome()

# 開啟 Facebook
driver.get("https://www.facebook.com/")
time.sleep(3)

# 登入 Facebook
username = driver.find_element(By.ID, "email")
password = driver.find_element(By.ID, "pass")

username.send_keys("") #輸入帳號
password.send_keys("") #輸入密碼

login_button = driver.find_element(By.NAME, "login")
login_button.click()
time.sleep(5)

# 搜尋粉絲專頁
driver.get("")
time.sleep(5)

# 預設
SCROLL_PAUSE_TIME = 1
scroll = True
posts_text = ['']
last_len_posts = 0

# 爬文時間上限
minu = 20
max_duration = minu * 35
start_time = time.time()


try:
    while scroll:

        elapsed_time = time.time() - start_time
        if elapsed_time > max_duration:
            print("已達執行時間限制，停止爬取")
            break

        # 滾動頁面（超過2000可能會漏）
        driver.execute_script("window.scrollBy(0, 2000);")
        time.sleep(SCROLL_PAUSE_TIME)

        # 所有貼文
        try:
            posts = driver.find_elements(By.XPATH, "//div[@data-ad-preview='message']")
        except Exception as e:
            print(e)
            continue

        # 只留下最後十篇和新的篇數（+10是防止有漏掉的）
        len_posts = len(posts)
        if len_posts - last_len_posts > 0:
            cut = -(len_posts - last_len_posts + 10)
        else:
            cut = -10
        last_len_posts = len_posts

        # 留下最後幾篇點開‘查看更多’
        posts = posts[cut:]
        for post in posts:
            try:
                see_more = post.find_element(By.XPATH, ".//div[@role='button' and text()='查看更多']")
                driver.execute_script("arguments[0].click();", see_more)
                time.sleep(0.1)
            except:
                pass

        # 所有貼文 最後幾篇點開過‘查看更多’的
        try:
            opened_posts = driver.find_elements(By.XPATH, "//div[@data-ad-preview='message']")
        except Exception as e:
            print(e)
            continue

        # 留下最後幾篇爬內文
        opened_posts = opened_posts[cut:]
        for post in opened_posts:
            try:
                text = post.text
                if text not in posts_text and "…… 查看更多" not in text:
                    posts_text.append(text)
            except:
                pass


except Exception as e:
    print(e)

end_time = time.time()

data = {
    'name': "邱毅",
    'text': posts_text
}

df = pd.DataFrame(data)
print(df)

# 續寫
df.to_csv('1207_3.csv', mode='a', header=False, index=False)

print("爬文耗時(s):", end_time - start_time)

driver.quit()